# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
Based on the paper titled "Show and Tell: A Neural Image Caption Generator" i choose to have an Encoder - Decoder Architecture, we start by Encoder which is a **pre-trained** CNN with a classification task, then we remove the last layer and we add a fully connected layer, wish is the features encoder, then we concatinate it to an Decoder, which a **non-trained** LSTM which result each time a one-hot vector with more than 8000 value, the value are mapped to a dictionary of words predefined already. than we apply a arg-sigmoid to get the predicted world each time. 

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
I kept the parameters as they are:
- The task is to describe a photo so i don't think we can downsample the image more than 256x256, if we do more we will lose details
- i keept the image treatments as : RandomCrop, RandomHorizontalFlip, Normalize as they will be used as data augmentation and avoiding overfiting.
- as batch size i putted 32, it is within the recommended range.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
i started by basic values as 256 for embeded size, 512 for hidden size and 32 as batch size and depends on the result of the model i will decide to keep or change them

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** i used Adam optimiser as it fix well the overfiting problem.

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True     # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 10             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
# criterion = nn.MSELoss()

optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.63s)
creating index...


  0%|          | 779/414113 [00:00<00:58, 7047.33it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:44<00:00, 9292.94it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [6]:
import torch.utils.data as data
import time
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
# response = requests.request("GET", 
#                             "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                             headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
#             requests.request("POST", 
#                              "https://nebula.udacity.com/api/v1/remote/keep-alive", 
#                              headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
        
        ts = time.time()
        # Get training statistics.
        stats = '%d - Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (ts,epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

1561299368 - Epoch [1/10], Step [100/12942], Loss: 1.9723, Perplexity: 7.1873
1561299422 - Epoch [1/10], Step [200/12942], Loss: 1.8715, Perplexity: 6.49790
1561299530 - Epoch [1/10], Step [400/12942], Loss: 2.0310, Perplexity: 7.62166
1561299585 - Epoch [1/10], Step [500/12942], Loss: 1.6222, Perplexity: 5.06433
1561299639 - Epoch [1/10], Step [600/12942], Loss: 2.3281, Perplexity: 10.2589
1561299693 - Epoch [1/10], Step [700/12942], Loss: 1.7030, Perplexity: 5.49044
1561299747 - Epoch [1/10], Step [800/12942], Loss: 1.8576, Perplexity: 6.40821
1561299801 - Epoch [1/10], Step [900/12942], Loss: 1.8389, Perplexity: 6.28988
1561299855 - Epoch [1/10], Step [1000/12942], Loss: 1.7366, Perplexity: 5.6777
1561299909 - Epoch [1/10], Step [1100/12942], Loss: 1.9056, Perplexity: 6.72374
1561299963 - Epoch [1/10], Step [1200/12942], Loss: 2.0026, Perplexity: 7.40851
1561300017 - Epoch [1/10], Step [1300/12942], Loss: 2.0441, Perplexity: 7.72232
1561300072 - Epoch [1/10], Step [1400/12942], Loss

1561304945 - Epoch [1/10], Step [10400/12942], Loss: 1.7506, Perplexity: 5.75807
1561304999 - Epoch [1/10], Step [10500/12942], Loss: 1.6100, Perplexity: 5.00267
1561305053 - Epoch [1/10], Step [10600/12942], Loss: 1.7299, Perplexity: 5.64038
1561305107 - Epoch [1/10], Step [10700/12942], Loss: 2.7920, Perplexity: 16.3136
1561305161 - Epoch [1/10], Step [10800/12942], Loss: 1.8784, Perplexity: 6.54335
1561305215 - Epoch [1/10], Step [10900/12942], Loss: 2.0453, Perplexity: 7.73121
1561305269 - Epoch [1/10], Step [11000/12942], Loss: 1.6714, Perplexity: 5.31988
1561305323 - Epoch [1/10], Step [11100/12942], Loss: 2.0146, Perplexity: 7.49768
1561305377 - Epoch [1/10], Step [11200/12942], Loss: 1.8416, Perplexity: 6.30652
1561305432 - Epoch [1/10], Step [11300/12942], Loss: 1.8218, Perplexity: 6.18274
1561305486 - Epoch [1/10], Step [11400/12942], Loss: 1.7450, Perplexity: 5.72590
1561305539 - Epoch [1/10], Step [11500/12942], Loss: 1.8378, Perplexity: 6.28251
1561305593 - Epoch [1/10], S

1561310483 - Epoch [2/10], Step [7700/12942], Loss: 3.8439, Perplexity: 46.7083
1561310537 - Epoch [2/10], Step [7800/12942], Loss: 1.7289, Perplexity: 5.63458
1561310591 - Epoch [2/10], Step [7900/12942], Loss: 2.2490, Perplexity: 9.47782
1561310644 - Epoch [2/10], Step [8000/12942], Loss: 2.1308, Perplexity: 8.42132
1561310699 - Epoch [2/10], Step [8100/12942], Loss: 1.8629, Perplexity: 6.4424
1561310752 - Epoch [2/10], Step [8200/12942], Loss: 2.0010, Perplexity: 7.39668
1561310806 - Epoch [2/10], Step [8300/12942], Loss: 1.8956, Perplexity: 6.65654
1561310861 - Epoch [2/10], Step [8400/12942], Loss: 1.8088, Perplexity: 6.103388
1561310915 - Epoch [2/10], Step [8500/12942], Loss: 1.6271, Perplexity: 5.08922
1561310969 - Epoch [2/10], Step [8600/12942], Loss: 1.8338, Perplexity: 6.25730
1561311023 - Epoch [2/10], Step [8700/12942], Loss: 1.7696, Perplexity: 5.86863
1561311077 - Epoch [2/10], Step [8800/12942], Loss: 1.6414, Perplexity: 5.16223
1561311131 - Epoch [2/10], Step [8900/12

1561316016 - Epoch [3/10], Step [5000/12942], Loss: 1.9428, Perplexity: 6.97804
1561316069 - Epoch [3/10], Step [5100/12942], Loss: 1.5120, Perplexity: 4.53564
1561316124 - Epoch [3/10], Step [5200/12942], Loss: 1.6671, Perplexity: 5.29668
1561316178 - Epoch [3/10], Step [5300/12942], Loss: 1.8062, Perplexity: 6.08717
1561316232 - Epoch [3/10], Step [5400/12942], Loss: 1.6214, Perplexity: 5.06035
1561316286 - Epoch [3/10], Step [5500/12942], Loss: 2.1260, Perplexity: 8.38097
1561316340 - Epoch [3/10], Step [5600/12942], Loss: 1.8836, Perplexity: 6.57696
1561316394 - Epoch [3/10], Step [5700/12942], Loss: 1.9315, Perplexity: 6.89975
1561316449 - Epoch [3/10], Step [5800/12942], Loss: 1.8645, Perplexity: 6.45307
1561316503 - Epoch [3/10], Step [5900/12942], Loss: 2.0163, Perplexity: 7.51037
1561316557 - Epoch [3/10], Step [6000/12942], Loss: 1.8922, Perplexity: 6.63428
1561316611 - Epoch [3/10], Step [6100/12942], Loss: 1.7791, Perplexity: 5.92476
1561316665 - Epoch [3/10], Step [6200/12

1561321549 - Epoch [4/10], Step [2300/12942], Loss: 2.0633, Perplexity: 7.87167
1561321603 - Epoch [4/10], Step [2400/12942], Loss: 1.8098, Perplexity: 6.10952
1561321657 - Epoch [4/10], Step [2500/12942], Loss: 1.8761, Perplexity: 6.52829
1561321711 - Epoch [4/10], Step [2600/12942], Loss: 1.7312, Perplexity: 5.64737
1561321765 - Epoch [4/10], Step [2700/12942], Loss: 1.6832, Perplexity: 5.38280
1561321819 - Epoch [4/10], Step [2800/12942], Loss: 1.6508, Perplexity: 5.21104
1561321873 - Epoch [4/10], Step [2900/12942], Loss: 1.7628, Perplexity: 5.82866
1561321927 - Epoch [4/10], Step [3000/12942], Loss: 1.9727, Perplexity: 7.19033
1561321981 - Epoch [4/10], Step [3100/12942], Loss: 1.8549, Perplexity: 6.39121
1561322034 - Epoch [4/10], Step [3200/12942], Loss: 1.7942, Perplexity: 6.01471
1561322089 - Epoch [4/10], Step [3300/12942], Loss: 1.9989, Perplexity: 7.38117
1561322143 - Epoch [4/10], Step [3400/12942], Loss: 1.8120, Perplexity: 6.12268
1561322196 - Epoch [4/10], Step [3500/12

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1561361846 - Epoch [9/10], Step [12100/12942], Loss: 1.4776, Perplexity: 4.3824
1561361900 - Epoch [9/10], Step [12200/12942], Loss: 1.6104, Perplexity: 5.0050
1561361954 - Epoch [9/10], Step [12300/12942], Loss: 1.8422, Perplexity: 6.31061
1561362008 - Epoch [9/10], Step [12400/12942], Loss: 1.8214, Perplexity: 6.18066
1561362063 - Epoch [9/10], Step [12500/12942], Loss: 1.8401, Perplexity: 6.29719
1561362117 - Epoch [9/10], Step [12600/12942], Loss: 1.8159, Perplexity: 6.14694
1561362171 - Epoch [9/10], Step [12700/12942], Loss: 1.8623, Perplexity: 6.43848
1561362225 - Epoch [9/10], Step [12800/12942], Loss: 2.0363, Perplexity: 7.66256
1561362279 - Epoch [9/10], Step [12900/12942], Loss: 1.8764, Perplexity: 6.52981
1561362356 - Epoch [10/10], Step [100/12942], Loss: 1.8397, Perplexity: 6.294876
1561362410 - Epoch [10/10], Step [200/12942], Loss: 1.8346, Perplexity: 6.26254
1561362463 - Epoch [10/10], Step [300/12942], Loss: 1.7236, Perplexity: 5.60492
1561362517 - Epoch [10/10], Step

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1561367114 - Epoch [10/10], Step [8900/12942], Loss: 1.5975, Perplexity: 4.9405
1561367168 - Epoch [10/10], Step [9000/12942], Loss: 1.9653, Perplexity: 7.13680
1561367222 - Epoch [10/10], Step [9100/12942], Loss: 1.7867, Perplexity: 5.96957
1561367277 - Epoch [10/10], Step [9200/12942], Loss: 1.6380, Perplexity: 5.14498
1561367330 - Epoch [10/10], Step [9300/12942], Loss: 1.7385, Perplexity: 5.68898
1561367385 - Epoch [10/10], Step [9400/12942], Loss: 1.6544, Perplexity: 5.23004
1561367439 - Epoch [10/10], Step [9500/12942], Loss: 1.9233, Perplexity: 6.84372
1561367493 - Epoch [10/10], Step [9600/12942], Loss: 1.7393, Perplexity: 5.69321
1561367547 - Epoch [10/10], Step [9700/12942], Loss: 1.5400, Perplexity: 4.66470
1561367601 - Epoch [10/10], Step [9800/12942], Loss: 1.7277, Perplexity: 5.62752
1561367655 - Epoch [10/10], Step [9900/12942], Loss: 1.8834, Perplexity: 6.57580
1561367709 - Epoch [10/10], Step [10000/12942], Loss: 1.7853, Perplexity: 5.9615
1561367763 - Epoch [10/10], S

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.